## Apply 메소드 활용

: 함수 브로드캐스팅시 한 번에 데이터프레임의 각 행과 열에 적용해서 실행 가능 

In [1]:
import pandas as pd

df = pd.DataFrame({'a':[10,20,30],'b':[20,30,40]})

print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [6]:
def my_sq(x):
    return x**2

def my_exp(x,n):
    return x ** n

sq = df['a'].apply(my_exp,n=2)
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


In [7]:
df = pd.DataFrame({'a':[10,20,30],'b':[20,30,40]})

In [8]:
def print_me(x):
    print(x)

print(df.apply(print_me,axis=0)) #열 방향 : 0, 행 방향:1

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object


In [10]:
# 인자가 여러개인 함수는 함수가 열 단위로 데이터를 처리할 수 있도록 처리
def avg_3_apply(col):
    x= col[0]
    y= col[1]
    z= col[2]
    return (x+y+z)/3

print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [11]:
# 행 방향으로 데이터 처리 

def avg_2_apply(row):
    sum = 0 
    for item in row:
        sum+= item
    return sum/df.shape[1]

print(df.apply(avg_2_apply, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


#### 데이터프레임의 누락값을 처리한 다음 apply 메서드 활용하기

In [12]:
#타이타닉 데이터 집합 사용
import seaborn as sns
titanic = sns.load_dataset('titanic')

print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [13]:
import numpy as np
#누락값 개수를 반환하는 count_missing 함수
def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

cmis_col = titanic.apply(count_missing)
print(cmis_col)
    

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [14]:
#누락값 비율계산하는 prop_missing
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [15]:
#데이터 비율 구하기
def prop_complete(vec):
    return 1 - prop_missing(vec)

In [17]:
#행방향으로 데이터 프레임 누락값 처리하기
cmis_row = titanic.apply(count_missing,axis=1)
pmis_row = titanic.apply(prop_missing,axis=1)
pcom_row = titanic.apply(prop_complete,axis =1)

print(cmis_row.head())
print(pmis_row.head())
print(pcom_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64
0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64
0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [19]:
#누락값 개수 데이터프레임에 추가
titanic['num_missing']=titanic.apply(count_missing,axis=1)

print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [21]:
print(titanic.loc[titanic.num_missing > 1,:].sample(10))

     survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
46          0       3    male   NaN      1      0  15.5000        Q  Third   
425         0       3    male   NaN      0      0   7.2500        S  Third   
198         1       3  female   NaN      0      0   7.7500        Q  Third   
878         0       3    male   NaN      0      0   7.8958        S  Third   
497         0       3    male   NaN      0      0  15.1000        S  Third   
301         1       3    male   NaN      2      0  23.2500        Q  Third   
61          1       1  female  38.0      0      0  80.0000      NaN  First   
95          0       3    male   NaN      0      0   8.0500        S  Third   
335         0       3    male   NaN      0      0   7.8958        S  Third   
573         1       3  female   NaN      0      0   7.7500        Q  Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
46     man        True  NaN   Queenstown    no  False            2  
425

### Group 연산

: 데이터를 집계하거나 변환하는 등의 작업을 한번에 처리할 수 있는 함수

#### 데이터 집계 - Groupby 함수

In [23]:
import pandas as pd
df = pd.read_csv('data/gapminder.tsv', sep='\t')

In [24]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


#### agg 메서드로 사용자 함수와 groupby 메서드 조합하기

In [25]:
#열의 평균값을 구하는 함수
def my_mean(values):
    n = len(values)
    sum=0
    for value in values:
        sum+=value
    
    return sum/n

In [27]:
#agg 함수로 사용자 함수 my_mean 적용
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [31]:
# 2개의 인자값을 받아 처리하는 사용자 함수와 groupby 메서드
def my_mean_diff(values,diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum/n
    return mean - diff_value

In [32]:
global_mean = df.lifeExp.mean()
print(global_mean)

agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff,diff_value=global_mean)
print(agg_mean_diff)

59.47443936619713
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


In [34]:
#집계 메서드를 리스트, 딕셔너리에 담아 전달하기
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero,np.mean,np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


In [36]:
gdf_dict = df.groupby('year').agg({'lifeExp':'mean','pop':'median','gdpPercap':'median'})
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371109


### 데이터 변환

In [39]:
#표준점수 계산
def my_zscore(x):
    return ((x-x.mean())/x.std())

#### transform( ): groupby 문에서 사용자 함수 적용

In [40]:
transform_z = df.groupby('year').lifeExp.transform(my_zscore)

print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


#### 누락값을 평균 값으로 처리하기

In [43]:
import seaborn as sns
import numpy as np

np.random.seed(42)

tips_10= sns.load_dataset('tips').sample(10)

tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill']=np.NaN
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [44]:
#성별로 그룹화 하여 평균값 구하기
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [45]:
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [47]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)

tips_10['fill_total_bill']=total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


### 데이터 필터링: filter()

In [49]:
tips =sns.load_dataset('tips')
print(tips.shape)
print(tips['size'].value_counts())

(244, 7)
2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [50]:
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count()>=30)

In [51]:
print(tips_filtered)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[231 rows x 7 columns]


### 그룹 오브젝트

: groupby 메소드가 반환하는 group object 

In [53]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [55]:
grouped = tips_10.groupby('sex')
print(grouped)

In [56]:
#groups: group오브젝트의 속성 출력
print(grouped.groups)

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'), 'Female': Int64Index([198, 124, 101], dtype='int64')}


In [58]:
#grouped 데이터 평균 구하기 - 계산 불가 데이터는 제외하고 구함
avgs = grouped.mean()
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [59]:
#get_group : 특정 조건의 group 오브젝트만 추출
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [60]:
#그룹오브젝트를 for 문에서 다루면 튜플로 반환 
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)
